<!-- dom:TITLE: Demo - Integration of functions -->
# Demo - Integration of functions
<!-- dom:AUTHOR: Mikael Mortensen Email:mikaem@math.uio.no at Department of Mathematics, University of Oslo. -->
<!-- Author: -->  
**Mikael Mortensen** (email: `mikaem@math.uio.no`), Department of Mathematics, University of Oslo.

Date: **Aug 7, 2020**

Copyright 2020, Mikael Mortensen. Released under CC Attribution 4.0 license

**Summary.** This is a demonstration of how the Python module [shenfun](https://github.com/spectralDNS/shenfun) can be used to
integrate over 2D surfaces in 3D space. We make use of
curvilinear coordinates, and reproduce the integrals
from [Chebfun](http://www.chebfun.org/examples/approx3/SurfaceIntegral3D.html).











## Introduction

A lesser known fact about [shenfun](https://github.com/spectralDNS/shenfun) is
that it can be used to perform regular, unweighted, integrals with
spectral accuracy. With the newly added curvilinear coordinates
we can now also integrate over highly complex lines and surfaces
embedded in 3D space.

To integrate over a domain in shenfun we use the
[inner](https://shenfun.readthedocs.io/en/latest/shenfun.forms.html#shenfun.forms.inner.inner)
function, with a constant test function. For
example, if we create some function space on the line from
0 to 1, then we can get this length using `inner`

In [1]:
from shenfun import *
B = FunctionSpace(10, 'C', domain=(0, 1))
length = inner(1, Array(B, val=1))
print('Length of domain =', length)

Length of domain = 1.0000000000000002


Not very impressive, but the same goes for multidimensional
tensor product domains

In [2]:
F = FunctionSpace(10, 'F', domain=(0, 2*np.pi))
T = TensorProductSpace(comm, (B, F))
area = inner(1, Array(T, val=1))
print('Area of domain =', area)

Area of domain = 6.283185307179587


Still not very impressive, but moving to curvilinear coordinates
it all starts to become more interesting.

Consider a 3D function $f(x,y,z) \in \mathbb{R}^3$ and
a 2D surface (not neccessarily plane) $S(u, v)$,
parametrized in two new coordinates $u$ and $v$. A position
vector $\mathbf{r}$ can be used to parametrize $S$

$$
\mathbf{r} = x(u, v) \,\mathbf{i} + y(u, v) \,\mathbf{j} + z(u, v) \,\mathbf{k},
$$

where $\mathbf{i}, \mathbf{j}, \mathbf{k}$ are the Cartesian unit vectors.
The two new coordinates $u$ and $v$ are functions of $x, y, z$,
and they each have a one-dimensional domain

$$
u \in D_u \quad v \in D_v.
$$

The exact size of the domain depends on the problem at hand. The computational
domain of the surface $S$ is $D=D_u \times D_v$.

A surface integral of $f$ over $S$ can now be written

$$
\int_S f(x, y, z) dxdydz = \int_D f(x(u, v), y(u, v), z(u, v)) \left|\frac{\partial \mathbf{r}}{\partial u} \times \frac{\partial \mathbf{r}}{\partial v} \right| dudv
$$

With shenfun such integrals are trivial, even for highly complex
domains.

## Example 1

Consider first the surface integral of $f(x,y,z)=x^2$
over the unit sphere. We use regular spherical coordinates,

$$
\begin{align*}
0 &\gt \theta \gt \pi \\ 
0 &\gt \phi \gt 2\pi \\ 
x(\theta, \phi) &= \sin \theta \cos \phi \\ 
y(\theta, \phi) &= \sin theta \sin \phi \\ 
z(\theta, \phi) &= \cos theta
\end{align*}
$$

The straight forward implementation of a function space for
the unit sphere reads

In [3]:
import sympy as sp

theta, phi = psi =sp.symbols('x,y', real=True, positive=True)
rv = (sp.sin(theta)*sp.cos(phi), sp.sin(theta)*sp.sin(phi), sp.cos(theta))

B0 = FunctionSpace(0, 'C', domain=(0, np.pi))
B1 = FunctionSpace(0, 'F', dtype='d')
T = TensorProductSpace(comm, (B0, B1), coordinates=(psi, rv, sp.Q.positive(sp.sin(theta))))

We can now approximate the function $f$ on this surface

In [4]:
f = Array(T, buffer=rv[0]**2)

and we can integrate over $S$

In [5]:
I = inner(1, f)

and finally compare to the exact result, which is $4 \pi / 3$

In [6]:
print('Error =', abs(I-4*np.pi/3))

Error = 7.459207029825166e-08


Note that we can here achieve better accuracy by using
more quadrature points. For example by refining `f`

In [7]:
T = T.get_refined(2*np.array(f.global_shape))
f = Array(T, buffer=rv[0]**2)
print('Error =', abs(inner(1, f)-4*np.pi/3))

Error = 0.0


Not bad at all:-)

## Example 2

Next, we solve [Example 5](http://www.math24.net/surface-integrals-of-first-kind.html)
from the online resources at math24.net. Here

$$
f = \sqrt{1+x^2+y^2}
$$

and the surface is defined by

$$
\mathbf{r} = u \cos v \mathbf{i} + u \sin v \mathbf{j} + v \mathbf{k}
$$

with $0 \gt u \gt 2, 0 \gt v 2\pi$.

The implementation is only a few lines, and we end by comparing
to the exact solution $14 \pi /3$

In [8]:
u, v = psi =sp.symbols('x,y', real=True, positive=True)
rv = (u*sp.cos(v), u*sp.sin(v), v)
B0 = FunctionSpace(0, 'C', domain=(0, 2))
B1 = FunctionSpace(0, 'C', domain=(0, np.pi))
T = TensorProductSpace(comm, (B0, B1), coordinates=(psi, rv))
f = Array(T, buffer=sp.sqrt(1+rv[0]**2+rv[1]**2))
print('Error =', abs(inner(1, f)-14*np.pi/3))

Error = 1.7763568394002505e-15


## Example 3

In this third example we use a surface that
looks like a seashell. Again, the example is taken from
[chebfun](http://www.chebfun.org/examples/approx3/SurfaceIntegral3D.html).

The surface of the seashell is parametrized with position
vector

$$
\begin{align*}
\mathbf{r} &= \left(\left(\frac{5}{4}-\frac{5 v}{8 \pi}\right)  \cos 2v(1+\cos u) + \cos 2v \right) \mathbf{i} \\ 
  &+\left(\left(\frac{5}{4}-\frac{5 v}{8 \pi}\right) \sin 2v (1+\cos u) + \sin 2v \right) \mathbf{j},\\ 
  &+\left(\frac{10 v}{2 \pi} + \left(\frac{5}{4}-\frac{5 v}{8 \pi}\right) \sin u + 15\right) \mathbf{k}
\end{align*}
$$

for $0 \gt u \gt 2 \pi, -2 \pi \gt v \gt 2 \pi$.

The function $f$ is now defined as

$$
f(x,y,z) = x+y+z
$$

The implementation is

In [9]:
rv = (5*(1-v/(2*sp.pi))*sp.cos(2*v)*(1+sp.cos(u))/4 + sp.cos(2*v),
      5*(1-v/(2*sp.pi))*sp.sin(2*v)*(1+sp.cos(u))/4 + sp.sin(2*v),
      10*v/(2*sp.pi) + 5*(1-v/(2*sp.pi))*sp.sin(u)/4 + 15)

B0 = FunctionSpace(100, 'C', domain=(0, 2*np.pi))
B1 = FunctionSpace(100, 'C', domain=(-2*np.pi, 2*np.pi))
T = TensorProductSpace(comm, (B0, B1), coordinates=(psi, rv, sp.Q.positive(v-2*sp.pi)))

f = rv[0]+rv[1]+rv[2]
fb = Array(T, buffer=f)
I = inner(1, fb)
print(I)

-6030.788147086101


which agrees very well with chebfun's result. We can plot the
seashell using mayavi. Unfortunately mayavi is a little tricky
to use with readthedocs, so instead of generating the plot
with the code below, we link here instead to a previously
generated image.

        from mayavi import mlab
        xx, yy, zz = T.local_curvilinear_mesh()
        mlab.figure(bgcolor=(1, 1, 1), size=(400, 400))
        mlab.mesh(xx, yy, zz, scalars=fb, colormap='jet')
        mlab.show()


<!-- dom:FIGURE: [https://cdn.jsdelivr.net/gh/spectralDNS/spectralutilities@master/figures/seashell3.png] -->
<!-- begin figure -->

<p></p>
<img src="https://cdn.jsdelivr.net/gh/spectralDNS/spectralutilities@master/figures/seashell3.png" >

<!-- end figure -->


<!-- ======= Bibliography ======= -->